In [1]:
#Read data
import cvxpy as cvx
import numpy as np
import math
#import mosek
from Linearization import *
case ="case33bw" #case33bw, case18ieee, case69, case141
[data, slack_bus, r, x, I, G, B, MVAlim, status] = data_import(case)

the following parameters are set up: data, slack_bus, r, x, I, G, B, MVAlim, status


In [2]:
#[a,b,c,v] =generate_rational_coeff(1e-7)
#print(v)
#print(approximation_error_of_BenTal(v))
#[a,b,c,v] =generate_rational_coeff(approximation_error_of_BenTal(v))
#print(v)
approximation_para =20
epsilon = approximation_error_of_BenTal(approximation_para)
print(epsilon)
[a,b,c,v] =generate_rational_coeff(epsilon)
print("equivalent number of rational constraints:", v)
coeff = [a,b,c,v]

1.1219913886861832e-12
equivalent number of rational constraints: 21


In [3]:
# Distribution power flows of Rabih A. Jabr
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
R = cvx.Variable(data['L'],nonneg=True) 
T = cvx.Variable(data['L'])            
v     = cvx.Variable(I) 
Isq  = cvx.Variable(data['L'],nonneg=True)

bus_Pinjection = [-sum(P[i,j]  for j in range(I) if connex(i,j,data)) for i in range(I)]
bus_Qinjection = [-sum(Q[i,j]  for j in range(I) if connex(i,j,data)) for i in range(I)]

Pgrid = -bus_Pinjection[slack_bus]*data['Sbase']
Qgrid = -bus_Qinjection[slack_bus]*data['Sbase']
    
const = []
const += [v[i] >= (data['Vmin'][i])**2 for i in range(I) if i != slack_bus]
const += [v[i] <= (data['Vmax'][i])**2 for i in range(I) if i != slack_bus]
const += [v[slack_bus] == 1]
 
dlmp_bus_index1=len(const)   
const += [(bus_Pinjection[i]+data['gs'][i]*v[i])== data['Pd'][i]/data['Sbase']  for i in range(I) if i != slack_bus]
dlmp_bus_index2= len(const)   

const += [(bus_Qinjection[i]+data['bs'][i]*v[i]) == data['Qd'][i]/data['Sbase'] for i in range(I) if i != slack_bus] 

for line in range(data['L']):
    i = data['from_bus'][line]-1
    j = data['to_bus'][line]-1       
    if status[line] ==1:
        const += [P[i,j] == (r[line]*v[i] - r[line]*R[line]+ x[line]*T[line])/(r[line]**2 + x[line]**2)]
        const += [Q[i,j] == (x[line]*v[i] - x[line]*R[line]- r[line]*T[line])/(r[line]**2 + x[line]**2)]
        const += [P[j,i] == (r[line]*v[j] - r[line]*R[line]- x[line]*T[line])/(r[line]**2 + x[line]**2) ]
        const += [Q[j,i] == (x[line]*v[j] - x[line]*R[line]+ r[line]*T[line])/(r[line]**2 + x[line]**2) ]        
        const += [Isq[line]  == (v[i]+v[j] - 2*R[line])/(r[line]**2 + x[line]**2) ]
        #repalce MVAlim
#        const += [(MVAlim[line]/data['Sbase']) >= cvx.norm2(cvx.bmat([[P[i,j], Q[i,j]]]))   ]
        # by constraint on current limits
#        const += [Isq[line] <= 4**2   ]
#        const += [(MVAlim[line]/data['Sbase'])**2 >= cvx.square(P[i,j]) + cvx.square(Q[i,j])]
     
rotated_cones =[]
lin_rotated_cones =[]
rational_rotated_cones =[]  
index1 = len(const)
for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
#        const += [cvx.quad_over_lin(cvx.bmat([[R[line], T[line]]]), v[i]) <= v[j]]
#        const += [cvx.norm2(cvx.bmat([[ R[line], T[line], 0.5*(v[i] - v[j])  ]] ))  <=  0.5*(v[i]+ v[j]) ]
#        const += [cvx.norm2(cvx.bmat([[ R[line], T[line], 0.5*(v[i] - v[j])  ]] ))  <=  0.5*(v[i]+ v[j]) ]

        rotated_cones += rotatedCONE(R[line], T[line], v[i], v[j])
        lin_rotated_cones += linearized_rotated_Cone(R[line], T[line], v[i], v[j], approximation_para)
        rational_rotated_cones += Rational_linearized_rotated_Cone(R[line], T[line], v[i], v[j], coeff)
index2 = len(rotated_cones)
          
prob = cvx.Problem(cvx.Minimize(Pgrid), const + rotated_cones) 
prob.solve(solver='MOSEK', warm_start = True) 
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print('loss:', Pgrid.value-sum(data['Pd']), 'MW')
print(prob.solver_stats.solve_time)

prob = cvx.Problem(cvx.Minimize(Pgrid), const + lin_rotated_cones) 
prob.solve(solver='CPLEX', warm_start = True) 
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print('loss:', Pgrid.value-sum(data['Pd']), 'MW')
print(prob.solver_stats.solve_time)

prob = cvx.Problem(cvx.Minimize(Pgrid), const + rational_rotated_cones) 
prob.solve(solver='CPLEX',  warm_start = True) 
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print('loss:', Pgrid.value-sum(data['Pd']), 'MW')
print(prob.solver_stats.solve_time)


#prob.solve(solver='CPLEX', verbose=True, warm_start = True) 
"""
import mosek
prob.solve(solver = 'MOSEK', 
                    mosek_params = {#mosek.dparam.optimizer_max_time:  100.0,
                                    mosek.iparam.intpnt_solve_form:   mosek.solveform.dual},
                    warm_start = True,
                    verbose = True)
"""

print('loss:', Pgrid.value-sum(data['Pd']), 'MW')

#print('objective'obj)
#print('voltages:', [math.sqrt(v[i].value) for i in range(I)])

print('== Active Power Flow == (in pu)')

"""
for line in range(data['L']):
    if data['status'][line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        print('P', 'line from bus', i, '-->', j, ': ', P[i][j].value)

print('== Reactive Power Flow == (in pu)')

for line in range(data['L']):
    if data['status'][line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        print('Q', 'line from bus', i, '-->', j, ': ', Q[i][j].value)    
              
print('We now check if the Second order conic constraints are binding:')
print('SOCP dual:', [const[i].dual_value for i in range(index1,index2)])
print ('All dual variables are positive so the relaxation is excat')

print('We now show DLMP:')
print('DLMP:', [const[i].dual_value for i in range(dlmp_bus_index1, dlmp_bus_index2)])

#Isq = [[0 for j in range(I)] for i in range(I)]
for line in range(data['L']):
    if data['status'][line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        print('current', 'line from bus', i, '-->', j, ': ', math.sqrt(Isq[i][j].value))  
"""

#print('We now check if the Second order conic constraints are binding:')
#print('SOCP dual:', [const[i].dual_value for i in range(index1,index2)])
#print ('If all dual variables are positive, the relaxation is excat')

print(" BUS DATA")
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus    Voltage")
for i in range(I):
    print(i+1, '  ', round(math.sqrt(v[i].value), 5))
print('================================================================================')    
for line in range(data['L']):   
    if status[line] ==1:
        print(math.sqrt(Isq.value[line]))
print('=================================END CVXPY==============================')
print('================================================================================')
#print(MATPOWER_RESULT)

imported active power from main grid: 3.917321503172957 MW
imported reactive power from main grid: 2.434922403262334 MVAr
loss: 0.20232150317295572 MW
0.046
imported active power from main grid: 3.9176771113119457 MW
imported reactive power from main grid: 2.4351409612131647 MVAr
loss: 0.20267711131194455 MW
0.5310000000172295
imported active power from main grid: 3.917678689325612 MW
imported reactive power from main grid: 2.4351420360292764 MVAr
loss: 0.20267868932561095 MW
0.9060000000172295
loss: 0.20267868932561095 MW
== Active Power Flow == (in pu)
 BUS DATA
imported active power from main grid: 3.917678689325612 MW
imported reactive power from main grid: 2.4351420360292764 MVAr
Bus    Voltage
1    1.0
2    0.99703
3    0.98294
4    0.97546
5    0.96806
6    0.94966
7    0.94617
8    0.94133
9    0.93506
10    0.92924
11    0.92838
12    0.92688
13    0.92077
14    0.9185
15    0.91709
16    0.91572
17    0.9137
18    0.91309
19    0.9965
20    0.99293
21    0.99222
22    0.99158

In [ ]:
#Branch flow model of Farivar and S. Low
const =[]
#data = spio.loadmat('Radial_Netdata', squeeze_me=True)
#r = data['r']
#x = data['x']
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
Vsq  = cvx.Variable(I)
Isq  = cvx.Variable(shape=(data['L'],), nonneg=True)

P_outflow_of = [ sum(P[j, data['to_bus'][line]-1] for line in range(data['L']) 
                  if  ((status[line]==1) and (j==data['from_bus'][line]-1)))  for j in range(I)]     

Q_outflow_of = [ sum(Q[j, data['to_bus'][line]-1] for line in range(data['L']) 
                  if  ((status[line]==1) and (j==data['from_bus'][line]-1)) ) for j in range(I)] 

P_inflow_of  = [  sum(P[data['from_bus'][line]-1,j] - r[line]*Isq[line] 
                        for line in range(data['L']) if  ((status[line]==1) and (j==data['to_bus'][line]-1)) ) 
                        for j in range(I)]  

Q_inflow_of  = [  sum(Q[data['from_bus'][line]-1,j] - x[line]*Isq[line] 
                        for line in range(data['L'])  if  ((status[line]==1) and (j==data['to_bus'][line]-1)))  
                        for j in range(I)] 

index1 = len(const)
for line in  range(data['L']): 
    if status[line]==1: 
        const +=rotatedCONE(P[data['from_bus'][line]-1,data['to_bus'][line]-1], Q[data['from_bus'][line]-1,data['to_bus'][line]-1], Vsq[data['from_bus'][line]-1],Isq[line] )

index2 = len(const)
for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        const +=[ Vsq[j] == Vsq[i] - 2*(r[line]*P[i,j]+x[line]*Q[i,j])+ (r[line]**2 + x[line]**2)*Isq[line] ]
#       replace aprperant line flows
#        const += [(MVAlim[line]/data['Sbase']) >= cvx.norm2(cvx.bmat( [[ P[i,j], Q[i,j] ]])) ]  
#       by line current limits (4pu) as Lucient Lobo
#        const += [Isq[line] <= 4**2 ] 
const +=[ Vsq[i] >= (data['Vmin'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[i] <= (data['Vmax'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[slack_bus] == 1]


dlmp_bus_index1 = len(const)
const += [data['gs'][j]*Vsq[j]  -(P_outflow_of[j] - P_inflow_of[j]) == data['Pd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]
dlmp_bus_index2 = len(const)

const += [data['bs'][j]*Vsq[j] -(Q_outflow_of[j] - Q_inflow_of[j]) == data['Qd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]

Pgrid = P_outflow_of[slack_bus]*data['Sbase']
Qgrid = Q_outflow_of[slack_bus]*data['Sbase']  
  
prob = cvx.Problem(cvx.Minimize(Pgrid), const) 

#prob = Problem(Minimize(obj*data['Sbase']), const) 
prob.solve(solver='MOSEK', verbose=True, warm_start = True) # method ='dccp', max_iter = 10, tau=100, ccp_times=1, mu=0.5 ) # method ='dccp', max_iter = 10, solver = 'SCS' )
 
print('imported grid:', Pgrid.value, 'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')

print('loss:', Pgrid.value-sum(data['Pd']))   

print('voltage:', [math.sqrt(Vsq[i].value) for i in range(I)])
"""
print('== Active Power Flow == (in pu)')

for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('P', 'line', i, '-->', j, ': ', P.value[i,j])

print('== Reactive Power Flow == (in pu)')
      
for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('Q', 'line', i, '-->', j, ': ', Q.value[i,j])      
"""            
print ('We now check if the Second order conic constraints are binding:')
print('SOCP dual:', [const[i].dual_value for i in range(index1,index2)])
print ('If all dual variables are positive, the relaxation is excat')

print('We now show DLMP:')
print('DLMP:', [-const[i].dual_value for i in range(dlmp_bus_index1, dlmp_bus_index2)])

#Check with Rah model
print(" BUS DATA")
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus    Voltage")
for i in range(I):
    print(i+1, '  ', round(math.sqrt(Vsq[i].value), 5))


In [15]:
#Branch flow model of Farivar and S. Low (Cleaner version)
const =[]
data = spio.loadmat('Radial_Netdata', squeeze_me=True)
r = data['r']
x = data['x']
P = cvx.Variable((I,I))
Q = cvx.Variable((I,I))
S = cvx.Variable(shape=(data['L'],), nonneg=True)
Vsq  = cvx.Variable(I)
Isq  = cvx.Variable(shape=(data['L'],), nonneg=True)

P_outflow_of = [0 for i in range(I)]
Q_outflow_of = [0 for i in range(I)]
P_inflow_of  = [data['gs'][i]*Vsq[i] for i in range(I)]
Q_inflow_of  = [data['bs'][i]*Vsq[i] for i in range(I)]

for j in range(I):
    for k in range(I):
        for line in range(data['L']):
            if status[line]==1:
                if (j==data['from_bus'][line]-1) and (k==data['to_bus'][line]-1):
                    P_outflow_of[j] +=  P[j,k]
                    Q_outflow_of[j] +=  Q[j,k]                   
for j in range(I):
    for i in range(I):
        for line in range(data['L']):
            if status[line]==1:
                if (i==data['from_bus'][line]-1) and (j==data['to_bus'][line]-1):
                    P_inflow_of[j] +=  (P[i,j] - r[line]*Isq[line])
                    Q_inflow_of[j] +=  (Q[i,j] - x[line]*Isq[line])                       

index1 = len(const)
for line in  range(data['L']): 
    if status[line]==1:
        const +=[ cvx.norm2(cvx.bmat( [[S[line], 0.5*(Vsq[data['from_bus'][line]-1]-Isq[line])]] ))
                                <= 0.5*(Vsq[data['from_bus'][line]-1]+Isq[line]) ]  

index2 = len(const)
const += [MVAlim[line]/data['Sbase'] >= S[line]]
for line in range(data['L']):
    if status[line] ==1:
        i = data['from_bus'][line]-1
        j = data['to_bus'][line]-1
        const +=[ Vsq[j] == Vsq[i] - 2*(r[line]*P[i,j]+x[line]*Q[i,j])+ (r[line]**2 + x[line]**2)*Isq[line] ]
        const += [S[line] >= cvx.norm2(cvx.bmat( [[ P[i,j], Q[i,j] ]])) ]  
#
const +=[ Vsq[i] >= (data['Vmin'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[i] <= (data['Vmax'][i])**2 for i in range(I) if i!=slack_bus]
const +=[ Vsq[slack_bus] == 1]

dlmp_bus_index1 = len(const)
const += [ -(P_outflow_of[j] - P_inflow_of[j]) == data['Pd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]
dlmp_bus_index2 = len(const)

const += [ -(Q_outflow_of[j] - Q_inflow_of[j]) == data['Qd'][j]/data['Sbase'] for j in range(I) if j !=slack_bus]

Pgrid = P_outflow_of[slack_bus]*data['Sbase']
Qgrid = Q_outflow_of[slack_bus]*data['Sbase']  

prob = cvx.Problem(cvx.Minimize(Pgrid), const) 

#prob = Problem(Minimize(obj*data['Sbase']), const) 
#prob.solve( solver='SCS',  warm_start = True, max_iters= 100000000 ) #verbose=True,
prob.solve( solver='MOSEK',  warm_start = True) #verbose=True,

# method ='dccp', max_iter = 10, tau=100, ccp_times=1, mu=0.5 ) # method ='dccp', max_iter = 10, solver = 'SCS' )
 
print('imported grid:', Pgrid.value, 'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')

print('loss:', Pgrid.value-sum(data['Pd']))   

print('voltage:', [math.sqrt(Vsq[i].value) for i in range(I)])

"""
print('== Active Power Flow == (in pu)')
for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('P', 'line', i, '-->', j, ': ', P.value[i,j])

print('== Reactive Power Flow == (in pu)')     
for i in range(I):
    for j in range(I):
        if connex[i,j]==1:
            print('Q', 'line', i, '-->', j, ': ', Q.value[i,j])      
"""  

print ('We now check if the Second order conic constraints are binding:')
print('SOCP dual:', [const[i].dual_value for i in range(index1,index2)])
print ('If all dual variables are positive, the relaxation is excat')

print('We now show DLMP:')
print('DLMP:', [-const[i].dual_value for i in range(dlmp_bus_index1, dlmp_bus_index2)])

#Check with Rah model
print(" BUS DATA")
print('imported active power from main grid:', Pgrid.value,'MW')
print('imported reactive power from main grid:', Qgrid.value,'MVAr')
print("Bus    Voltage")
for i in range(I):
    print(i+1, '  ', round(math.sqrt(Vsq[i].value), 5))
    
for line in range(data['L']):   
    if status[line] ==1:
        print(math.sqrt(Isq.value[line]))
print('=================================END CVXPY==============================')
print('================================================================================')

imported grid: 3.676403546293312 MW
imported reactive power from main grid: 2.3492570910653527 MVAr
loss: 0.18640354629331046
voltage: [0.9816304800881109, 0.9815523162045384, 0.9810716083839551, 0.9808928268879745, 0.9806356335114318, 0.9804120464665536, 0.9684930631901812, 0.9598292974491455, 0.9593148758961981, 0.9586173590537219, 0.9585029956982174, 0.9597265154104198, 0.9473161447225118, 0.9587316935386461, 0.9468298561402627, 0.9464824499835738, 0.9466677668325857, 0.9355383764711984, 0.9350694255486264, 0.9347645666265766, 0.9345607335193895, 0.9340678960937034, 0.9338625865261246, 0.9333458804298067, 0.9332227683277103, 0.9328951570805655, 0.932766443414548, 0.9329437009506925, 0.9326646297693937, 0.9325018405618581, 0.9326836133565045, 0.9325425754083188, 0.9322961830782586, 0.9469688126662412, 0.9317569426786213, 0.9311474515260713, 0.9308648647533986, 0.9309119535081174, 0.9307588694688168, 0.9295816647708052, 0.9294283615823666, 0.9274672886898482, 0.9269943385918058, 0.925